In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [2]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [3]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [4]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [5]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [53]:
cancer='BRCA'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [86]:
train

array([[ 0.00874834, -0.00736022, -0.01710605, ...,  0.0059139 ,
         0.00585509,  0.01760065],
       [-0.06756009, -0.03635654, -0.06104614, ...,  0.00184041,
         0.01073223, -0.00660949],
       [ 0.05017487, -0.03385318, -0.05728117, ..., -0.01067972,
         0.01398897,  0.01295414],
       ...,
       [-0.00012984,  0.00158047,  0.00504004, ...,  0.00526114,
        -0.00421565, -0.0168382 ],
       [-0.0120549 ,  0.0223974 , -0.02588812, ..., -0.03524445,
         0.00812885,  0.01039054],
       [ 0.00757404,  0.00548185,  0.02304116, ...,  0.00104388,
         0.00138618, -0.0349668 ]])

In [87]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [18]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=1
scoring='roc_auc'
#np.random.seed(42)
random_states=np.random.choice(range(101), loopn, replace=False)

tmp_train=[]
tmp_test=[]
#print(random_states)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )
    print("train_x.shpae:")
    print(train_X.shape)

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
    estimator = PCA(n_components=0.95,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)

    n_components=range(10,min(pca_X_train.shape),10)
    print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=10)
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),
                                     param_grid=parameters,cv=cvx,scoring=scoring)
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000)
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                         LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),
                                         LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                            scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])
    print("n_components:")
    print(n_components[np.argmax(best_pca_train_scores)])
    
    tmp_train.append(best_pca_train_scores)
    tmp_test.append(best_pca_test_scores)

train_x.shpae:
(195, 13022)
range(10, 132, 10)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.79777778
 0.79777778 0.79778056 0.79778056 0.79778056        nan        nan
 0.5               nan 0.5        0.80732387 0.80732387 0.80944027
 0.80944027 0.80944027        nan        nan 0.78772765        nan
 0.78772765 0.80894737 0.80894737 0.81001392 0.80948482 0.80948482
        nan        nan 0.80202172        nan 0.80202172 0.81582568
 0.81582568 0.81529936 0.81529936 0.81477304        nan        nan
 0.81265664        nan 0.81370927 0.8121359  0.8121359  0.81319131
 0.8121359  0.812665          nan        nan 0.81477304        nan
 0.81318853 0.8131941  0.8131941  0.8131941  0.812665   0.812665
        nan        nan 0.8131941         nan 0.812665   0.8137232
 0.8137232  0.8137232  0.8131941  0.812665          nan        nan
 0.8131941         nan 0.812665   0.8137232  0.8137232  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.8183765
 0.8183765  0.8183765  0.8183765  0.8183765         nan        nan
 0.5               nan 0.5        0.85062935 0.85062935 0.85116124
 0.85116124 0.85168755        nan        nan 0.78772765        nan
 0.78772765 0.88390977 0.88390977 0.88813422 0.88866332 0.88866332
        nan        nan 0.80785297        nan 0.80679476 0.90397661
 0.90397661 0.90399053 0.90557226 0.90451685        nan        nan
 0.87653857        nan 0.88234754 0.89500139 0.89500139 0.88972988
 0.89079087 0.89976608        nan        nan 0.87811473        nan
 0.90188527 0.88971596 0.88971596 0.88340295 0.89184907 0.89924255
        nan        nan 0.87496519        nan 0.89924534 0.88602896
 0.88550265 0.87759955 0.89184907 0.89924255        nan        nan
 0.87018379        nan 0.89871345 0.88549986 0.8860289

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.81836814
 0.81836814 0.81784461 0.81784461 0.81784461        nan        nan
 0.5               nan 0.5        0.8522083  0.8522083  0.85432749
 0.85432749 0.85379838        nan        nan 0.78772765        nan
 0.78772765 0.88499025 0.88499025 0.88711779 0.88711779 0.88711779
        nan        nan 0.80679476        nan 0.80679476 0.89238931
 0.89238931 0.89239488 0.89239488 0.89239209        nan        nan
 0.8591061         nan 0.87549986 0.88552214 0.88552214 0.8897633
 0.89028683 0.89345586        nan        nan 0.86018658        nan
 0.89028683 0.88235032 0.88235032 0.88553885 0.89028126 0.89292398
        nan        nan 0.85120301        nan 0.89397939 0.87971039
 0.87971039 0.87602896 0.88975494 0.89292676        nan        nan
 0.83211083        nan 0.89292398 0.87812865 0.8776023

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


n_components:
110
Wall time: 1min 43s


In [19]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))
pd.DataFrame(tmp_train)

mean
0.8376952300548755
0.7829059829059828
std
0.0
0.0


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,"0 0.754221 Name: AUC, dtype: float64","0 0.809941 Name: AUC, dtype: float64","0 0.805087 Name: AUC, dtype: float64","0 0.827406 Name: AUC, dtype: float64","0 0.797119 Name: AUC, dtype: float64","0 0.811102 Name: AUC, dtype: float64","0 0.795958 Name: AUC, dtype: float64","0 0.805667 Name: AUC, dtype: float64","0 0.826826 Name: AUC, dtype: float64","0 0.833421 Name: AUC, dtype: float64","0 0.837695 Name: AUC, dtype: float64","0 0.817697 Name: AUC, dtype: float64","0 0.822552 Name: AUC, dtype: float64"


In [106]:
print(train_scores)
print(test_scores)

[0.9295183982683983]
[0    0.940972
Name: AUC, dtype: float64]


In [ ]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )
print("train_x.shpae:")
print(train_X.shape)

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
estimator = PCA(n_components=0.95,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)

n_components=range(10,min(pca_X_train.shape),10)
print(n_components)
    #n_components=[0.99,0.95,0.90,0.85]
best_pca_train_scores=[]
best_pca_test_scores=[]
for j in n_components:
    estimator = PCA(n_components=j,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    
        #scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

    svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),
                                 param_grid=parameters,cv=cvx,scoring=scoring)
    svc_grid_search.fit(pca_X_train, train_y)

    param_grid = {'penalty':['l1', 'l2'],
                  "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                  "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                  }
    LR_grid = LogisticRegression(max_iter=1000, random_state=10)
    LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
    LR_grid_search.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]
    clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5, random_state=10),
                             n_jobs=10)
    clf.fit(pca_X_train, train_y)

    estimators = [
        ('lr', LR_grid_search.best_estimator_),
        ('svc', svc_grid_search.best_estimator_),
    ]

    param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                    LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                    LogisticRegression(C=0.1),LogisticRegression(C=1),
                                    LogisticRegression(C=10),LogisticRegression(C=100),
                                    LogisticRegression(C=1000)]}

    Stacking_grid =StackingClassifier(estimators=estimators,)
    Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                        scoring=scoring,n_jobs=10)
    Stacking_grid_search.fit(pca_X_train, train_y)
    Stacking_grid_search.best_estimator_

    train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)

    test_pre_y = Stacking_grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
        
    best_pca_train_scores.append(train_res1.loc[:,"AUC"])
    best_pca_test_scores.append(test_res1.loc[:,"AUC"])

train_x.shpae:
(643, 4660)
range(10, 225, 10)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.7143901
 0.7143901  0.71497091 0.71497091 0.71497091        nan        nan
 0.5               nan 0.5        0.71448764 0.71448764 0.71269409
 0.71288869 0.71283911        nan        nan 0.75544363        nan
 0.75544363 0.7410049  0.7410049  0.71928803 0.72452302 0.72069172
        nan        nan 0.72603681        nan 0.72980901 0.77485496
 0.77485496 0.73512124 0.72827393 0.72121291        nan        nan
 0.76355163        nan 0.72094655 0.78226702 0.78226702 0.76229474
 0.7282634  0.72160058        nan        nan 0.78163126        nan
 0.72194098 0.78337635 0.78337635 0.78181982 0.72840992 0.7217952
        nan        nan 0.78313284        nan 0.72184103 0.78327786
 0.78322902 0.78332596 0.72879609 0.72169752        nan        nan
 0.78318092        nan 0.72174562 0.78327709 0.78327709

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73744721
 0.73744721 0.73763951 0.73763951 0.73763951        nan        nan
 0.5               nan 0.5        0.7433069  0.74325806 0.74151261
 0.74156293 0.74146606        nan        nan 0.75544363        nan
 0.75544363 0.77454046 0.77454046 0.75697895 0.75959677 0.75761014
        nan        nan 0.73242755        nan 0.73732497 0.79365361
 0.79370245 0.74703995 0.74829754 0.75203096        nan        nan
 0.78937715        nan 0.74698316 0.80836777 0.80836777 0.7623341
 0.74498145 0.74987725        nan        nan 0.81378871        nan
 0.74993    0.81038702 0.81043512 0.79692278 0.74464253 0.74958343
        nan        nan 0.81042987        nan 0.74968263 0.81013757
 0.81018565 0.80859132 0.74493254 0.74953609        nan        nan
 0.81032918        nan 0.74968034 0.81013681 0.8101849

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73784327
 0.73784327 0.73847134 0.7384706  0.73856754        nan        nan
 0.5               nan 0.5        0.74743194 0.74743194 0.74569005
 0.74564274 0.74569012        nan        nan 0.75544363        nan
 0.75544363 0.77962244 0.77962244 0.7626641  0.76586647 0.76344024
        nan        nan 0.73228179        nan 0.73703563 0.78971182
 0.78966298 0.74765061 0.75118777 0.7558488         nan        nan
 0.78436332        nan 0.75332094 0.7899935  0.7899935  0.73979204
 0.7468031  0.75439466        nan        nan 0.78859021        nan
 0.75371177 0.77913378 0.77913378 0.75611042 0.74665508 0.75410307
        nan        nan 0.77289331        nan 0.75419923 0.77110017
 0.77110017 0.77085519 0.74636276 0.75376341        nan        nan
 0.77032685        nan 0.75390919 0.77032685 0.770278

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.73774223
 0.73774223 0.73793537 0.73798345 0.7378873         nan        nan
 0.5               nan 0.5        0.7468876  0.7468876  0.74456428
 0.74485357 0.74475441        nan        nan 0.75544363        nan
 0.75544363 0.77703875 0.77703875 0.75920547 0.76158358 0.76037243
        nan        nan 0.73233063        nan 0.73713183 0.77770512
 0.77770512 0.72906771 0.74151393 0.75070816        nan        nan
 0.7741012         nan 0.75750629 0.77093566 0.77088758 0.71042541
 0.73553939 0.74867026        nan        nan 0.77095301        nan
 0.75026571 0.7476316  0.74758276 0.71003318 0.73529593 0.74813677
        nan        nan 0.71200598        nan 0.74862216 0.71900639
 0.71905449 0.70854622 0.73519973 0.74833139        nan        nan
 0.69955503        nan 0.74837947 0.70788092 0.707880

In [52]:
print(np.max(best_pca_train_scores))
print(best_pca_test_scores[np.argmax(best_pca_train_scores)])
print("n_components:")
print(n_components[np.argmax(best_pca_train_scores)])

print(best_pca_train_scores)
print(best_pca_test_scores)

0.94004329004329
0    0.958333
Name: AUC, dtype: float64
n_components:
140
[0    0.827624
Name: AUC, dtype: float64, 0    0.890611
Name: AUC, dtype: float64, 0    0.890611
Name: AUC, dtype: float64, 0    0.899702
Name: AUC, dtype: float64, 0    0.916126
Name: AUC, dtype: float64, 0    0.902733
Name: AUC, dtype: float64, 0    0.5
Name: AUC, dtype: float64, 0    0.91756
Name: AUC, dtype: float64, 0    0.930952
Name: AUC, dtype: float64, 0    0.92059
Name: AUC, dtype: float64, 0    0.926488
Name: AUC, dtype: float64, 0    0.5
Name: AUC, dtype: float64, 0    0.911661
Name: AUC, dtype: float64, 0    0.940043
Name: AUC, dtype: float64, 0    0.895238
Name: AUC, dtype: float64, 0    0.5
Name: AUC, dtype: float64, 0    0.901299
Name: AUC, dtype: float64, 0    0.885038
Name: AUC, dtype: float64]
[0    0.854167
Name: AUC, dtype: float64, 0    0.878472
Name: AUC, dtype: float64, 0    0.868056
Name: AUC, dtype: float64, 0    0.923611
Name: AUC, dtype: float64, 0    0.902778
Name: AUC, dtype: float6